# generate ground truth pose from reconstructed camera poses + manually found world pose

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import json
from pathlib import Path
from typing import List

import cv2
import dataclass_array as dca
import jax.numpy as jnp
import matplotlib.pyplot as plt
import mitsuba as mi
import numpy as np
from PIL import Image
import pycolmap
import pyrender
import transforms3d as t3d
import trimesh
import visu3d as v3d

import burybarrel.colmap_util as cutil
from burybarrel.image import render_v3d

In [3]:
# manually find from blender
### barrelddt1 ###
scalefactor = 0.2
t_gt = np.array([-0.160665, -0.063128, 3.60226])
blender_eulers = np.array([-49.6539, 0, 0]) * np.pi / 180
### barrel4 ###
# scalefactor = 0.68
# t_gt = np.array([-3.3115, 1.55458, 3.30907])
# blender_eulers = np.array([223.25, -11.152, 147.46]) * np.pi / 180

R_gt = t3d.euler.euler2mat(*blender_eulers)
T_gt = np.eye(4)
T_gt[:3, :3] = R_gt
T_gt[:3, 3] = t_gt
T_gt = v3d.Transform.from_matrix(T_gt)

In [ ]:
datadir = Path("/scratch/jeyan/barreldata/divedata/dive8/barrelddt1")
resdir = Path("/scratch/jeyan/barreldata/results/barrelddt1")
obj_path = Path("/scratch/jeyan/barreldata/models3d/barrelsingle-scaled.ply")

reconstr_path = resdir / "colmap-out/0"
imgdir = datadir / "rgb"

reconstruction = pycolmap.Reconstruction(reconstr_path)
print(reconstruction.summary())
imgpaths = sorted(list(imgdir.glob("*.png")) + list(imgdir.glob("*.jpg")))
imgs = np.array([cv2.imread(imgpath) for imgpath in imgpaths])

In [5]:
p, c = cutil.get_pc(reconstruction)
pts3d = v3d.Point3d(p=p, rgb=c)
cams = cutil.get_cams_v3d(reconstruction)
# v3d.make_fig([cams, pts3d])

In [6]:
mesh = trimesh.load(obj_path)
vtxs = np.array(mesh.vertices)
rgb = np.zeros_like(vtxs, dtype=np.uint8)
rgb[:, 0] = 255
vtxs_p3d = v3d.Point3d(p=vtxs, rgb=rgb)

In [ ]:
# scaled 3d
scaleT = v3d.Transform.from_matrix(np.eye(4) * scalefactor)
tofig = []
camscaled = cams.replace(world_from_cam=cams.world_from_cam.replace(t=cams.world_from_cam.t * scalefactor))
sceneptsscaled = pts3d.apply_transform(scaleT)
tofig.extend([sceneptsscaled, camscaled, T_gt @ vtxs_p3d])
v3d.make_fig(*tofig)

In [8]:
gtoverlaydir = resdir / "gt-overlays"
gtoverlaydir.mkdir(exist_ok=True)
for i, img in enumerate(imgs):
    imgpath = imgpaths[i]
    Image.fromarray(render_v3d(camscaled[i], T_gt @ vtxs_p3d, radius=4, background=img)).save(gtoverlaydir / f"{imgpaths[i].stem}.png")

In [9]:
obj2cams_truth = [cam.world_from_cam.inv @ T_gt for cam in camscaled]
gt_data_list = []
for i, T in enumerate(obj2cams_truth):
    truthdata = {
        "img_path": str(imgpaths[i]),
        "img_id": str(i),
        "R": T.R.tolist(),
        "t": T.t.tolist(),
    }
    gt_data_list.append(truthdata)
with open(datadir / "gt_obj2cam.json", "wt") as f:
    json.dump(gt_data_list, f)